# Michael Nguyen

## Naive Bayes on Political Text

In this notebook we use Naive Bayes to explore and classify political data. See the `README.md` for full details. You can download the required DB from the shared dropbox or from blackboard

In [5]:
import sqlite3
import nltk
import random
import numpy as np
from collections import Counter, defaultdict

import pandas as pd
from nltk.corpus import stopwords
from string import punctuation
import re
import collections

In [14]:
# Cleaning Script
sw = stopwords.words("english")
punctuation = set(punctuation) 
tw_punct = punctuation - {"#", "'"}

def remove_stop(text, sw=sw) :
    # modify this function to remove stopwords
    return([ch for ch in text if ch not in sw])


def remove_punctuation(text, punct_set=tw_punct) : 
   text = [''.join(ch for ch in word if ch not in punct_set)
           for word in text]
   
   return(text)


def tokenize(text) : 
    """ Splitting on whitespace rather than the book's tokenize function. That 
        function will drop tokens like '#hashtag' or '2A', which we need for Twitter. """
    
    # modify this function to return tokens
    text = text.strip().split()
    return(text)

def clean_tokenize(text):
    # Remove punctuations
    text = ''.join(ch for ch in text if ch not in punctuation)
    
    # Convert text to lower case and split into words
    text = text.lower().split()
    
    # Remove stop words
    text = [word for word in text if word not in sw]

    return text

In [15]:
convention_db = sqlite3.connect("C:/Users/micha/applied text mining/mod 4/2020_Conventions.db")
convention_cur = convention_db.cursor()

In [16]:
convention_cur.execute("SELECT name FROM sqlite_master WHERE type='table';")

# Get all results
tables = convention_cur.fetchall()

# See table
for table in tables:
    print(table[0])

conventions


### Part 1: Exploratory Naive Bayes

We'll first build a NB model on the convention data itself, as a way to understand what words distinguish between the two parties. This is analogous to what we did in the "Comparing Groups" class work. First, pull in the text 
for each party and prepare it for use in Naive Bayes.  

In [18]:
convention_data = []

# fill this list up with items that are themselves lists. The 
# first element in the sublist should be the cleaned and tokenized
# text in a single string. The second element should be the party. 

query_results = convention_cur.execute(
    '''
    SELECT text, party FROM conventions
    '''
)

for row in query_results:
    text, party = row
    text = clean_tokenize(text)
    convention_data.append([text, party])

Let's look at some random entries and see if they look right. 

In [19]:
random.choices(convention_data,k=10)

[[['never',
   'raised',
   'way',
   'would',
   'running',
   'public',
   'office',
   'didn’t',
   'interest',
   'graduated',
   'college',
   'got',
   'married',
   'five',
   'children',
   'six',
   'years',
   'life',
   'children',
   'grown',
   'opportunity',
   'run',
   'congress',
   'came',
   'along'],
  'Democratic'],
 [['work',
   'hard',
   'contribute',
   'community',
   'pay',
   'taxes',
   'i’ve',
   'gone',
   'school',
   'built',
   'good',
   'life',
   'two',
   'daughters'],
  'Democratic'],
 [['joe', 'biden’s', 'granddaughter', 'interview', 'take', 'one'],
  'Democratic'],
 [['internationally',
   'we’ve',
   'turned',
   'back',
   'agreements',
   'forged',
   'husband',
   'alliances',
   'championed',
   'presidents',
   'like',
   'reagan',
   'eisenhower',
   'home',
   'george',
   'floyd',
   'brianna',
   'taylor',
   'never',
   'ending',
   'list',
   'innocent',
   'people',
   'color',
   'continue',
   'murdered',
   'stating',
   'simple'

If that looks good, we now need to make our function to turn these into features. In my solution, I wanted to keep the number of features reasonable, so I only used words that occur at least `word_cutoff` times. Here's the code to test that if you want it. 

In [21]:
word_cutoff = 5

tokens = [w for t, p in convention_data for w in t]

word_dist = nltk.FreqDist(tokens)

feature_words = set()

for word, count in word_dist.items() :
    if count > word_cutoff :
        feature_words.add(word)
        
print(f"With a word cutoff of {word_cutoff}, we have {len(feature_words)} as features in the model.")

With a word cutoff of 5, we have 2391 as features in the model.


In [22]:
def conv_features(text,fw):
    ret_dict = dict()
    #split text
    words = text.split()
    for word in words:
        if word in fw:
            ret_dict[word] = True
    return ret_dict

In [23]:
assert(len(feature_words)>0)
assert(conv_features("donald is the president",feature_words)==
       {'donald':True,'president':True})
assert(conv_features("people are american in america",feature_words)==
                     {'america':True,'american':True,"people":True})

Now we'll build our feature set. Out of curiosity I did a train/test split to see how accurate the classifier was, but we don't strictly need to since this analysis is exploratory. 

In [25]:
def conv_features(text,fw):
    #marking fw
    ret_dict = dict()
    for word in text: 
        if word in fw:
            ret_dict[word] = True
    return ret_dict

In [26]:
featuresets = [(conv_features(text,feature_words), party) for (text, party) in convention_data]

In [27]:
random.seed(20220507)
random.shuffle(featuresets)

test_size = 500

In [28]:
test_set, train_set = featuresets[:test_size], featuresets[test_size:]
classifier = nltk.NaiveBayesClassifier.train(train_set)
print(nltk.classify.accuracy(classifier, test_set))

0.5


In [29]:
classifier.show_most_informative_features(25)

Most Informative Features
                   china = True           Republ : Democr =     25.8 : 1.0
                   votes = True           Democr : Republ =     23.8 : 1.0
             enforcement = True           Republ : Democr =     21.5 : 1.0
                 destroy = True           Republ : Democr =     19.2 : 1.0
                freedoms = True           Republ : Democr =     18.2 : 1.0
                 climate = True           Democr : Republ =     17.8 : 1.0
                supports = True           Republ : Democr =     17.1 : 1.0
                   crime = True           Republ : Democr =     16.1 : 1.0
                   media = True           Republ : Democr =     14.9 : 1.0
                 beliefs = True           Republ : Democr =     13.0 : 1.0
               countries = True           Republ : Democr =     13.0 : 1.0
                 defense = True           Republ : Democr =     13.0 : 1.0
                    isis = True           Republ : Democr =     13.0 : 1.0

Write a little prose here about what you see in the classifier. Anything odd or interesting?

### My Observations

The most informative features shows distinct focuses for each party, for example, 'climate' and 'votes' for Democrats and 'china' and 'trade' for Republicans. Also, there are more aggressive tone in Republican tweets with words like 'destroy' and 'enemy'. Words like 'liberal' appearing more in Republican tweets can mean that they are tweets referring to their political opponents.



## Part 2: Classifying Congressional Tweets

In this part we apply the classifer we just built to a set of tweets by people running for congress
in 2018. These tweets are stored in the database `congressional_data.db`. That DB is funky, so I'll
give you the query I used to pull out the tweets. Note that this DB has some big tables and 
is unindexed, so the query takes a minute or two to run on my machine.

In [32]:
cong_db = sqlite3.connect("C:/Users/micha/applied text mining/mod 4/congressional_data.db")
cong_cur = cong_db.cursor()

In [33]:
results = cong_cur.execute(
        '''
           SELECT DISTINCT 
                  cd.candidate, 
                  cd.party,
                  tw.tweet_text
           FROM candidate_data cd 
           INNER JOIN tweets tw ON cd.twitter_handle = tw.handle 
               AND cd.candidate == tw.candidate 
               AND cd.district == tw.district
           WHERE cd.party in ('Republican','Democratic') 
               AND tw.tweet_text NOT LIKE '%RT%'
        ''')

results = list(results) # Just to store it, since the query is time consuming

In [40]:
tweet_data = []

for candidate, party, tweet in results:
    # Decode byte-string
    decoded_tweet = tweet.decode('utf-8') 
    # Split tweet into words
    words = decoded_tweet.split() 
    tweet_data.append((words, party))

In [42]:
print(tweet_data[:5])

[(['"Brooks', 'Joins', 'Alabama', 'Delegation', 'in', 'Voting', 'Against', 'Flawed', 'Funding', 'Bill"', 'http://t.co/3CwjIWYsNq'], 'Republican'), (['"Brooks:', 'Senate', 'Democrats', 'Allowing', 'President', 'to', 'Give', 'Americans’', 'Jobs', 'to', 'Illegals"', '#securetheborder', 'https://t.co/mZtEaX8xS6'], 'Republican'), (['"NASA', 'on', 'the', 'Square"', 'event', 'this', 'Sat.', '11AM', '–', '4PM.', 'Stop', 'by', '&amp;', 'hear', 'about', 'the', 'incredible', 'work', 'done', 'in', '#AL05!', '@DowntownHSV', 'http://t.co/R9zY8WMEpA'], 'Republican'), (['"The', 'trouble', 'with', 'Socialism', 'is', 'that', 'eventually', 'you', 'run', 'out', 'of', 'other', "people's", 'money."', '-', 'Margaret', 'Thatcher', 'https://t.co/X97g7wzQwJ'], 'Republican'), (['"The', 'trouble', 'with', 'socialism', 'is', 'eventually', 'you', 'run', 'out', 'of', 'other', "people's", 'money"', '–', 'Thatcher.', "She'll", 'be', 'sorely', 'missed.', 'http://t.co/Z8gBnDQUh8'], 'Republican')]


In [43]:
party_counts = {}
for item in tweet_data:
    party = item[1]  # assuming the party is the second element of each sublist
    if party not in party_counts:
        party_counts[party] = 0
    party_counts[party] += 1
print(party_counts)

{'Republican': 288531, 'Democratic': 376125}


There are a lot of tweets here. Let's take a random sample and see how our classifer does. I'm guessing it won't be too great given the performance on the convention speeches...

In [44]:
random.seed(20201014)

tweet_data_sample = random.choices(tweet_data,k=10)

In [45]:
for tweet, party in tweet_data_sample :
    estimated_party = classifier.classify(conv_features(tweet, feature_words))
    
    print(f"Here's our (cleaned) tweet: {tweet}")
    print(f"Actual party is {party} and our classifer says {estimated_party}.")
    print("")
    

Here's our (cleaned) tweet: ['Earlier', 'today,', 'I', 'spoke', 'on', 'the', 'House', 'Floor', 'abt', 'protecting', 'health', 'care', 'for', 'women', 'and', 'praised', '@PPmarmonte', 'for', 'their', 'work', 'on', 'the', 'Central', 'Coast.', 'https://t.co/WqgTRzT7VV']
Actual party is Democratic and our classifer says Republican.

Here's our (cleaned) tweet: ['Go', 'Tribe!', '#RallyTogether', 'https://t.co/0NXutFL9L5']
Actual party is Democratic and our classifer says Democratic.

Here's our (cleaned) tweet: ['Apparently,', 'Trump', 'thinks', "it's", 'just', 'too', 'easy', 'for', 'students', 'overwhelmed', 'by', 'the', 'crushing', 'burden', 'of', 'debt', 'to', 'pay', 'off', 'student', 'loans', '#TrumpBudget', 'https://t.co/ckYQO5T0Qh']
Actual party is Democratic and our classifer says Republican.

Here's our (cleaned) tweet: ['We’re', 'grateful', 'for', 'our', 'first', 'responders,', 'our', 'rescue', 'personnel,', 'our', 'firefighters,', 'our', 'police,', 'and', 'volunteers', 'who', 'hav

Now that we've looked at it some, let's score a bunch and see how we're doing.

In [46]:
# dictionary of counts by actual party and estimated party. 
# first key is actual, second is estimated
parties = ['Republican','Democratic']
results = defaultdict(lambda: defaultdict(int))

for p in parties :
    for p1 in parties :
        results[p][p1] = 0


num_to_score = 10000
random.shuffle(tweet_data)

for idx, tp in enumerate(tweet_data) :
    tweet, party = tp    
    
    estimated_party = classifier.classify(conv_features(tweet, feature_words))
    
    results[party][estimated_party] += 1
    
    if idx > num_to_score : 
        break

In [47]:
results

defaultdict(<function __main__.<lambda>()>,
            {'Republican': defaultdict(int,
                         {'Republican': 3025, 'Democratic': 1253}),
             'Democratic': defaultdict(int,
                         {'Republican': 4087, 'Democratic': 1637})})

### Reflections

For the results what I can see is:
- For True Republicans: Out of tweets actually posted by Republicans, my classifier correctly identified 3025 as Republican, but it mistakenly classified 1253 as Democratic.
- For True Democrats: Out of tweets actually posted by Democrats, my classifier correctly identified 1637 as Democratic, but it mistakenly classified 4087 as Republican.